<a href="https://colab.research.google.com/github/SAHILLNEGII/SAHILLNEGII/blob/main/CNN_object_identification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install keras

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import os
import tensorflow_datasets as tfds

In [3]:
# Load COCO 2017 dataset
dataset, info = tfds.load('cifar100', split=['train', 'test'], with_info=True, as_supervised=True)

train_dataset, test_dataset = dataset[0], dataset[1]

# Check dataset information
print(info)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar100/incomplete.BJ1CZA_3.0.2/cifar100-train.tfrecord*...:   0%|       …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar100/incomplete.BJ1CZA_3.0.2/cifar100-test.tfrecord*...:   0%|        …

Dataset cifar100 downloaded and prepared to /root/tensorflow_datasets/cifar100/3.0.2. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='cifar100',
    full_name='cifar100/3.0.2',
    description="""
    This dataset is just like the CIFAR-10, except it has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).
    """,
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    data_dir=PosixGPath('/tmp/tmpqsmc7utetfds'),
    file_format=tfrecord,
    download_size=Unknown size,
    dataset_size=132.03 MiB,
    features=FeaturesDict({
        'coarse_label': ClassLabel(shape=(), dtype=int64, num_classes=20),
        'id': Text(shape=(), dtype=string),
        'image': Image(shape=(32, 32, 3), dtype=uint8),
        'label

In [4]:
IMAGE_HEIGHT = 480
IMAGE_WIDTH = 720
BATCH_SIZE = 32
NUM_CLASSES = 100  # Cifar has 100 classes

def preprocess(image, label):
    # Resize image and normalize
    image = tf.image.resize(image, [IMAGE_HEIGHT, IMAGE_WIDTH]) / 255.0
    # One-hot encode the label
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

train_dataset = train_dataset.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [5]:
# Load MobileNetV2 as the base model
base_model = MobileNetV2(weights='imagenet', include_top=False,input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3),pooling=None,classes=1000,classifier_activation="softmax")

# Freeze base model layers
base_model.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 480, 720, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 240, 360, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 240, 360, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (N

In [ ]:
EPOCHS = 10

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS
)

# Save the model
model.save('Classification_model_pretrained_Sahil.h5')

Epoch 1/10
1563/1563 [==============================] - 2994s 2s/step - loss: 4.1228 - accuracy: 0.0669 - val_loss: 3.6332 - val_accuracy: 0.1531
Epoch 2/10
1563/1563 [==============================] - 3088s 2s/step - loss: 3.7709 - accuracy: 0.1087 - val_loss: 3.4346 - val_accuracy: 0.1862
Epoch 3/10
1563/1563 [==============================] - 3053s 2s/step - loss: 3.6779 - accuracy: 0.1259 - val_loss: 3.3256 - val_accuracy: 0.1995
Epoch 4/10
1563/1563 [==============================] - 3068s 2s/step - loss: 3.6298 - accuracy: 0.1337 - val_loss: 3.3088 - val_accuracy: 0.2136
Epoch 5/10
 175/1563 [==>...........................] - ETA: 39:49 - loss: 3.6029 - accuracy: 0.1409

In [ ]:
# Unfreeze some layers in the base model
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Compile again with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
fine_tune_epochs = 10
total_epochs = EPOCHS + fine_tune_epochs

history_fine = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=fine_tune_epochs,
    initial_epoch=EPOCHS
)

# Save the fine-tuned model
model.save('Classification_model_finetuned_Sahil.h5')

In [ ]:
# Evaluate the fine-tuned model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Validation Accuracy: {test_accuracy:.2f}")